In [1]:
import os
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
from tqdm import tqdm
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np

In [2]:
# Parameters
image_size = 224
batch_size = 48
num_classes = 2  # maize and turmeric
learning_rate = 0.0001
num_epochs = 50

In [3]:
# Define data transformations
transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    transforms.Lambda(lambda x: x + 1.35 * torch.randn_like(x)),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [4]:
# Define dataset and data loaders
output_folder = './split-dataset-2'
train_dataset = datasets.ImageFolder(os.path.join(output_folder, 'train'), transform=transform)
val_dataset = datasets.ImageFolder(os.path.join(output_folder, 'val'), transform=transform)
test_dataset = datasets.ImageFolder(os.path.join(output_folder, 'test'), transform=transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [5]:
# Use GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [6]:
# Define CNN model
class SimpleCNN(nn.Module):
    def __init__(self, num_classes):
        super(SimpleCNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.classifier = nn.Sequential(
            nn.Linear(256 * (image_size // 8) * (image_size // 8), 1024),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5),
            nn.Linear(1024, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

In [7]:
# Initialize model
model = SimpleCNN(num_classes=num_classes)
model.to(device)

SimpleCNN(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Linear(in_features=200704, out_features=1024, bias=True)
    (1): ReLU(inplace=True)
    (2): Dropout(p=0.5, inplace=False)
    (3): Linear(in_features=1024, out_features=2, bias=True)
  )
)

In [8]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [9]:
def ant_colony_optimization(features, num_iterations=10, num_ants=20, evaporation_rate=0.5, pheromone_influence=1, heuristic_influence=2):
    num_features = features.shape[1]
    pheromone_levels = torch.ones(num_features, device=features.device)
    heuristic_values = torch.var(features, dim=0)

    for iteration in range(num_iterations):
        feature_counts = torch.zeros(num_features, device=features.device)

        for ant in range(num_ants):
            selected_features = []
            for feature_idx in range(num_features):
                probability = (pheromone_levels[feature_idx] ** pheromone_influence) * (heuristic_values[feature_idx] ** heuristic_influence)
                if torch.rand(1).item() < probability:
                    selected_features.append(feature_idx)
                    feature_counts[feature_idx] += 1

        pheromone_levels = (1 - evaporation_rate) * pheromone_levels + feature_counts / num_ants

    threshold = torch.mean(pheromone_levels)
    selected_features_indices = [i for i in range(num_features) if pheromone_levels[i] > threshold]
    selected_features = features[:, selected_features_indices]

    return selected_features

In [10]:
# Training the ACO-CNN model
val_accuracies = []
train_losses = []
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for images, labels in tqdm(train_loader, desc=f"Epoch {epoch + 1}/{num_epochs}"):
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = model(images)

        # Apply ACO to optimize features
        optimized_features = ant_colony_optimization(outputs)

        # Compute loss using optimized features (dummy usage here, should adapt to a real case)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    train_losses.append(avg_loss)
    print(f"Epoch {epoch + 1}, Training Loss: {avg_loss:.4f}")

    # Validation
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f"Validation Accuracy: {accuracy:.2f}%")
    val_accuracies.append(accuracy)
    
print("Training complete.")

Epoch 1/50: 100%|████████████████████████████████████████████████████████████████████| 411/411 [02:15<00:00,  3.03it/s]


Epoch 1, Training Loss: 0.4146
Validation Accuracy: 90.73%


Epoch 2/50: 100%|████████████████████████████████████████████████████████████████████| 411/411 [02:05<00:00,  3.28it/s]


Epoch 2, Training Loss: 0.2058
Validation Accuracy: 92.20%


Epoch 3/50: 100%|████████████████████████████████████████████████████████████████████| 411/411 [02:04<00:00,  3.29it/s]


...

Epoch 48, Training Loss: 0.0611
Validation Accuracy: 95.92%


Epoch 49/50: 100%|███████████████████████████████████████████████████████████████████| 411/411 [02:24<00:00,  2.85it/s]


Epoch 49, Training Loss: 0.0611
Validation Accuracy: 95.78%


Epoch 50/50: 100%|███████████████████████████████████████████████████████████████████| 411/411 [02:24<00:00,  2.85it/s]


Epoch 50, Training Loss: 0.0595
Validation Accuracy: 96.14%
Training complete.


In [11]:
import pandas as pd
val_df = pd.DataFrame({
    "Epoch": list(range(1, num_epochs + 1)),
    "Validation Accuracy": val_accuracies,
    "Training Loss": train_losses
})

# Append to a CSV file (ensure this file name is unique for each model)
val_df.to_csv("model-3_val_accuracy.csv", mode='a', index=False, header=not pd.io.common.file_exists("model-3_val_accuracy.csv"))

In [12]:
# Evaluation on test set
model.eval()
true_labels = []
predictions = []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs = inputs.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)

        true_labels.extend(labels.cpu().numpy())
        predictions.extend(predicted.cpu().numpy())

# Calculate evaluation metrics
accuracy = accuracy_score(true_labels, predictions)
precision = precision_score(true_labels, predictions, average='weighted')
recall = recall_score(true_labels, predictions, average='weighted')
f1 = f1_score(true_labels, predictions, average='weighted')

print(f"Test Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

Test Accuracy: 0.9607
Precision: 0.9599
Recall: 0.9607
F1 Score: 0.9601


In [13]:
# In each model notebook, at the end of your metrics calculation, do this:
import pandas as pd

# Example metrics for Model 1
aco_metrics = {
    "Model": "ACO-CNN",
    "Accuracy": accuracy,
    "Precision": precision,
    "Recall": recall,
    "F1 Score": f1,
}

metrics_df = pd.DataFrame([aco_metrics])

# Append to CSV file
metrics_df.to_csv("model_metrics-2.csv", mode='a', index=False, header=not pd.io.common.file_exists("model_metrics-2.csv"))
